# Créer et exporter une catégorie au format Moodle XML

Besoin de 
`pip install array-to-latex` et `pip install xmltodict`

In [3]:
# Permet a une cellule d'avoir plus d'un display en sortie
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

Technical functions:

In [6]:
from xml.dom.minidom import parseString
from xml.sax.saxutils import unescape
import xmltodict
import json
import io

In [7]:
def savestr(string, filename="new.txt", raw=False):
    if raw:
        string = string.replace('\t','') # no tabs
        string = string.replace('\n','') # no linebreak
    else:
        string = string.replace('\t','  ') # double space instead of tabs
    text_file = io.open(filename, "w", encoding='utf8') # essential for accents and other characters
    text_file.write(string)
    text_file.close()
    
def paragraph(string): 
    if string is "":
        return string
    else:
        return "<![CDATA[<p>\(\)" + string + "</p>]]>"  # \(\) pour activer latex dans Moodle

In [8]:
def set_oparg(name, default_value, **opargs):
    if name in opargs:
        return opargs.get(name)
    else:
        return default_value

### Création de Moodle XML

In [9]:
def create_category(name="stupid category name", description=""):
    """ Creates a Moodle category. So far only for the Test.
        name, str : name of the category
        description, str : description of the category
    """
    category = {
        "quiz": {
            "question": [
            {
            "@type": "category",
            "category": {"text": "$module$/top/" + name},
            "info": {"@format": "html", "text": paragraph(description)}
            }
            ]
        }
    }
    return category

In [10]:
def save_category(category, file_name):
    """ Save a category under the format Moodle XML """
    category_xml = xmltodict.unparse(category, pretty=True)
    savestr(unescape(category_xml), file_name + ".xml")

In [11]:
def create_question(content="Stupid question", name="", **opargs):
    """ Creates a Moodle question. There is a lot of possible options, not all suported yet.
        name, str : Name of the question
        content, str : Content of the question. Can include Latex (MathJax latex).
        Optional arguments : 
        grade, float : grade for the question. Default: 1    
        graderinfo, string : Information for the person grading (= the answer). Default: ""
    """
    grade = set_oparg('grade', 1.0, **opargs)
    graderinfo = set_oparg('graderinfo', "", **opargs)
    question = {
        "@type": "essay", 
        "name": {"text": name},
        "questiontext": {
            "@format": "html", 
            "text": paragraph(content)
        },
        "generalfeedback": {"@format": "html", "text": ""},
        "defaultgrade": grade,
        "penalty": "0.0000000",
        "hidden": "0",
        "idnumber": "",
        "responseformat": "editor",
        "responserequired": "1",
        "responsefieldlines": 5,
        "attachments": "0",
        "attachmentsrequired": "0",
        "graderinfo": {"@format": "html","text": paragraph(graderinfo)},
        "responsetemplate": {"@format": "html","text": ""}
    }
    return question

In [12]:
def append_question(category, question): # adds a question to a category
    category['quiz']['question'].append(question)

### Générer une question dans une catégorie et les envoyer dans Moodle

In [13]:
category_name = "ma super catégorie"
category_description = "osef"

category = create_category(category_name, category_description)

In [14]:
question_intitule = " Calculer la dérivée de $f(x) = e^x + \frac{1}{2} \Vert x \Vert^2$."
question_name = "Question de cours (dérivée)"
question_note = 3.0
correction = "$e^x + x$"

question = create_question(
            content=question_intitule,
            name=question_name,
            grade=question_note,
            graderinfo=correction)
append_question(category, question) # ajoute la question à la catégorie

In [15]:
# une deuxième pour la route
question = create_question(
            content="Combien font $2+2$?",
            name="arithmétique",
            grade=2.0,
            graderinfo="Réponse : $\sqrt{16}$")
append_question(category, question)

In [16]:
save_category(category, "ma_categorie")

La dernière commande crée un fichier `ma_categorie.xml` qu'il suffit d'importer dans Moodle.
Dans l'interface du cours, à droite, aller dans `banque de questions > importer ` puis 
- format de fichier : Format XML Moodle
- Généraux : Cocher "Obtenir la catégorie à partir du fichier", "Obtenir le contexte à partir du fichier" si ce n'est déjà fait
- Glisser le fichier
- Cliquer "importation"
- Si tout est bon (vert) valider encore (continuer).
Ensuite les questions sont prêtes à l'utilisation dans tous les tests.


### Générer plein de questions aléatoirement et les envoyer dans Moodle

In [44]:
import numpy as np
from numpy.random import randint
import array_to_latex as a2l
def strlatex(A): return a2l.to_ltx(A, frmt = '{:2.0f}', arraytype = 'pmatrix', print_out=False)

In [27]:
category = create_category("produit matriciel", "liste de questions sur le produit matriciel")

In [45]:
# on peut créer une matrice aléatoire et afficher son code latex comme un string
A = randint(-3,3,(3,3))
print(strlatex(A))

\begin{pmatrix}
 -3  & -1  & -1 \\
 -3  & -2  & -3 \\
   0  & -1  & -1 
\end{pmatrix}


In [47]:
# du coup on peut en mettre plein les questions
for k in range(10):
    A = randint(-3,3,(3,3))
    B  = randint(-3,3,(3,2))
    intitule = "Calculer le produit matriciel suivant : $$"+strlatex(A)+strlatex(B)+" $$"
    question = create_question(
                                content=intitule,
                                name="Produit matriciel",
                                grade=3.0,
                                graderinfo=strlatex(A@B)) # python nous donne la réponse 
    append_question(category, question)

In [48]:
save_category(category, "produit_matriciel")

Un peu de documentation:

- https://docs.moodle.org/3x/fr/Format_XML_Moodle
- https://docs.moodle.org/3x/fr/Questions
- https://docs.moodle.org/31/en/Embedded_files_repository
- AMC to Moodle : https://github.com/nennigb/amc2moodle
- GIFT to Moodle : https://www.lmspulse.com/2020/gift-format-moodle-quiz-scripting/
- Dict to XML : https://stackoverflow.com/questions/36021526/converting-an-array-dict-to-xml-in-python
    - bon ca marche pas terrible relou avec attribute et plein de trucs automatiques
- Dict to XML : https://github.com/martinblech/xmltodict
    - meilleur controle sur tout
    - par contre l'output est un string avec des tabs? Ok facile a resoudre
    - ne gère pas le CDATA : besoin de https://github.com/martinblech/xmltodict/issues/57 ? Yes